In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import sklearn.metrics

import random
random.seed(7)


def generator(X, Y, batch_size=32, train=True):
    while True:
        for offset in range(0, len(X), batch_size):
            X_batch = np.stack(X[offset:offset+batch_size], axis=0)
            
            if train:
                Y_batch = np.stack(Y[offset:offset+batch_size], axis=0)
            
                Y_batch_ = np.array(list(map(np.sum, Y_batch)))
                Y_batch_[Y_batch_ < 100] = 0
                Y_batch_[Y_batch_ >= 100] = 1
                

                Y_f = np.array([Y_batch_[e].flatten() for e in range(Y_batch_.shape[0])])
                yield (X_batch, Y_f)
            
            else:
                yield X_batch

structure_ids = []   
for line in open('./structures lists/stucture ids homo sapiens.txt', 'r'):
    line = line.strip('\n').lower()
    structure_ids.append(line)
for line in open('./structures lists/stucture ids synthetic construct.txt', 'r'):
    line = line.strip('\n').lower()
    structure_ids.append(line)
for line in open('./structures lists/stucture ids virus.txt', 'r'):
    line = line.strip('\n').lower()
    structure_ids.append(line)

structure_ids.remove('1a9n')
structure_ids.remove('2adc')
random.shuffle(structure_ids)
print(len(structure_ids))

num_test = int(len(structure_ids)*0.3)

X_train = []
X_test = []
Y_train = []
Y_test = []
num_aa_train = 0
num_aa_test = 0
for j, structure_id in enumerate(structure_ids):
    protein = np.load('../data/voxelized data 14x14x17 2/' + structure_id + '_protein.npy')
    rna = np.load('../data/voxelized data 14x14x17 2/' + structure_id + '_rna_3D.npy')
    
    na = 0
    pos = 0
    while (np.sum(rna[na]) > 100) and (na < len(rna)-1):
        pos +=1
        na +=1
    
    if pos == 0:
        continue

    if j <= num_test:
        if pos > len(rna)/2:
            X_test.extend(protein[:, :, :, :, :3])
            Y_test.extend(rna)
            num_aa_test +=len(rna)
        else:
            X_test.extend(protein[:pos, :, :, :, :3])
            X_test.extend(protein[-pos:, :, :, :, :3])
            Y_test.extend(rna[:pos])
            Y_test.extend(rna[-pos:])
            num_aa_test +=2*pos

    else:
        if pos > len(rna)/2:
            X_train.extend(protein[:, :, :, :, :3])
            Y_train.extend(rna)
            num_aa_train +=len(rna)
        else:
            X_train.extend(protein[:pos, :, :, :, :3])
            X_train.extend(protein[-pos:, :, :, :, :3])
            Y_train.extend(rna[:pos])
            Y_train.extend(rna[-pos:])
            num_aa_train +=2*pos


Y_test = np.stack(Y_test, axis=0)
Y_test_ = np.array(list(map(np.sum, Y_test)))
Y_test_[Y_test_ < 100] = 0
Y_test_[Y_test_ >= 100] = 1


print(Y_test.shape)

n_steps_train = int(num_aa_train/400) 
n_steps_test = int(num_aa_test/400)

print(num_aa_train, num_aa_test)


generator_train = generator(X_train, Y_train, 400, True)
generator_validation = generator(X_test, Y_test, 400, True)
generator_test = generator(X_test, Y_test, 400, False)

ins = tf.keras.layers.Input((14, 14, 17, 3))
con1 = tf.keras.layers.Conv3D(filters=64, kernel_size=(3, 3, 3), padding='same', activation='relu')(ins)
con2 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con1)
con3 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(con2)
maxp1 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con3)
con4 = tf.keras.layers.Conv3D(filters=32, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp1)
con5 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con4)
con6 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(con5)
maxp2 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con6)
con7 = tf.keras.layers.Conv3D(filters=16, kernel_size=(3, 3, 3), padding='same', activation='relu')(maxp2)
con8 = tf.keras.layers.Conv3D(filters=8, kernel_size=(3, 3, 3), padding='same', activation='relu')(con7)
con9 = tf.keras.layers.Conv3D(filters=4, kernel_size=(3, 3, 3), padding='same', activation='relu')(con8)
maxp3 = tf.keras.layers.MaxPool3D(pool_size=(2, 2, 2))(con9)
batch = tf.keras.layers.BatchNormalization()(maxp3)
flat = tf.keras.layers.Flatten()(batch)
dens2 = tf.keras.layers.Dense(units=256, activation='relu')(flat)
drop2 = tf.keras.layers.Dropout(0.6)(dens2)
outs = tf.keras.layers.Dense(units=1, activation='sigmoid')(drop2)
model = tf.keras.models.Model(inputs=ins, outputs=outs)
model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(lr=0.00001), metrics=['accuracy', 'mse'])

model.summary()

# checkpoint
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50, mode='min', min_delta=0.0001)
# mc = tf.keras.callbacks.ModelCheckpoint("weights_best.hdf5", monitor='val_loss', verbose=0, save_best_only=True, mode='min')

# model.fit(X_train, Y_train_f, validation_split=0.33, epochs=1, batch_size=200, callbacks=callbacks_list, verbose=0)
history = model.fit_generator(generator_train, steps_per_epoch=n_steps_train, epochs=500, 
                    validation_data = generator_validation, validation_steps=n_steps_test, callbacks=[es], verbose=1)

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.savefig('accuracy_new')
plt.figure()
plt.plot(epochs, loss, 'r', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.savefig('loss_new')


# model_best = model
# model_best.load_weights("weights_best.hdf5")
# print(model.evaluate(X_test, Y_test, verbose=0, batch_size=100))
# model_best.save('model_cnn_15_2.h5')
# Y_pred = model_best.predict(X_test, batch_size=200)
Y_pred_ = model.predict_generator(generator_test, steps=n_steps_test)
print(Y_pred_.shape)


#CNN
Y_pred_prob = Y_pred_
Y_pred_[Y_pred_ >= 0.5] = 1
Y_pred_[Y_pred_ < 0.5] = 0

print(Y_pred_.shape)
print(Y_test_.shape)
Y_test_ = Y_test_[:Y_pred_.shape[0]]


print('CNN: \n')
confusion_matrix = sklearn.metrics.confusion_matrix(Y_test_, Y_pred_)
accuracy = np.sum(np.trace(confusion_matrix))/np.sum(confusion_matrix)
auc = sklearn.metrics.roc_auc_score(Y_test_, Y_pred_prob)

print(confusion_matrix, np.round(accuracy, 2), np.round(auc, 2))

# baseline model
# predict all zeros; at least 50% correct predictions because there are 1/2 of negative examples
Y_pred_base = np.zeros(Y_test_.shape)

# po = np.sum(Y_train, axis=0)/Y_train.shape[0]
# po[po >= 0.5] = 1
# po[po < 0.5] = 0
# Y_pred_base = np.tile(po, (Y_test.shape[0],1))

print(f'\n BASELINE MODEL: \n')
confusion_matrix_base = sklearn.metrics.confusion_matrix(Y_test_, Y_pred_base)
accuracy_base = np.sum(np.trace(confusion_matrix_base))/np.sum(confusion_matrix_base)
auc_base = sklearn.metrics.roc_auc_score(Y_test_, Y_pred_base)

print(confusion_matrix_base, np.round(accuracy_base, 2), np.round(auc_base, 2))

#random model
Y_pred_random = np.random.random(Y_test_.shape)
Y_pred_random[Y_pred_random >= 0.5] = 1
Y_pred_random[Y_pred_random < 0.5] = 0

print(f'\n RANDOM MODEL: \n')
confusion_matrix_random = sklearn.metrics.confusion_matrix(Y_test_, Y_pred_random)
accuracy_random = np.sum(np.trace(confusion_matrix_random))/np.sum(confusion_matrix_random)
auc_random = sklearn.metrics.roc_auc_score(Y_test_, Y_pred_random)

print(confusion_matrix_random, np.round(accuracy_random, 2), np.round(auc_random, 2))

549
(24252, 14, 14, 17)
46405 24252
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 14, 14, 17, 3)     0         
_________________________________________________________________
conv3d (Conv3D)              (None, 14, 14, 17, 64)    5248      
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 14, 14, 17, 32)    55328     
_________________________________________________________________
conv3d_2 (Conv3D)            (None, 14, 14, 17, 32)    27680     
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 7, 7, 8, 32)       0         
_________________________________________________________________
conv3d_3 (Conv3D)            (None, 7, 7, 8, 32)       27680     
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 7, 7

116/116 [==============================] - 28s 238ms/step - loss: 0.6221 - acc: 0.6433 - mean_squared_error: 0.2166 - val_loss: 0.6426 - val_acc: 0.6272 - val_mean_squared_error: 0.2260
Epoch 29/500
116/116 [==============================] - 27s 236ms/step - loss: 0.6178 - acc: 0.6522 - mean_squared_error: 0.2147 - val_loss: 0.6441 - val_acc: 0.6324 - val_mean_squared_error: 0.2267
Epoch 30/500
116/116 [==============================] - 28s 237ms/step - loss: 0.6145 - acc: 0.6493 - mean_squared_error: 0.2132 - val_loss: 0.6415 - val_acc: 0.6414 - val_mean_squared_error: 0.2255
Epoch 31/500
116/116 [==============================] - 27s 235ms/step - loss: 0.6090 - acc: 0.6603 - mean_squared_error: 0.2108 - val_loss: 0.6396 - val_acc: 0.6462 - val_mean_squared_error: 0.2246
Epoch 32/500
116/116 [==============================] - 27s 236ms/step - loss: 0.6049 - acc: 0.6647 - mean_squared_error: 0.2089 - val_loss: 0.6343 - val_acc: 0.6513 - val_mean_squared_error: 0.2224
Epoch 33/500
116/1

116/116 [==============================] - 27s 236ms/step - loss: 0.4660 - acc: 0.7848 - mean_squared_error: 0.1518 - val_loss: 0.6081 - val_acc: 0.6770 - val_mean_squared_error: 0.2095
Epoch 70/500
116/116 [==============================] - 27s 236ms/step - loss: 0.4652 - acc: 0.7850 - mean_squared_error: 0.1515 - val_loss: 0.6105 - val_acc: 0.6750 - val_mean_squared_error: 0.2102
Epoch 71/500
116/116 [==============================] - 27s 236ms/step - loss: 0.4609 - acc: 0.7871 - mean_squared_error: 0.1500 - val_loss: 0.6076 - val_acc: 0.6759 - val_mean_squared_error: 0.2091
Epoch 72/500
116/116 [==============================] - 27s 236ms/step - loss: 0.4578 - acc: 0.7872 - mean_squared_error: 0.1488 - val_loss: 0.6071 - val_acc: 0.6763 - val_mean_squared_error: 0.2086
Epoch 73/500
116/116 [==============================] - 27s 235ms/step - loss: 0.4542 - acc: 0.7916 - mean_squared_error: 0.1473 - val_loss: 0.6100 - val_acc: 0.6742 - val_mean_squared_error: 0.2098
Epoch 74/500
116/1

116/116 [==============================] - 27s 235ms/step - loss: 0.3478 - acc: 0.8492 - mean_squared_error: 0.1094 - val_loss: 0.6384 - val_acc: 0.6764 - val_mean_squared_error: 0.2139
Epoch 111/500
116/116 [==============================] - 27s 235ms/step - loss: 0.3452 - acc: 0.8481 - mean_squared_error: 0.1085 - val_loss: 0.6311 - val_acc: 0.6846 - val_mean_squared_error: 0.2107
Epoch 112/500
116/116 [==============================] - 27s 235ms/step - loss: 0.3415 - acc: 0.8496 - mean_squared_error: 0.1073 - val_loss: 0.6342 - val_acc: 0.6833 - val_mean_squared_error: 0.2116
Epoch 113/500
116/116 [==============================] - 27s 235ms/step - loss: 0.3395 - acc: 0.8516 - mean_squared_error: 0.1065 - val_loss: 0.6348 - val_acc: 0.6815 - val_mean_squared_error: 0.2121
Epoch 114/500
116/116 [==============================] - 27s 236ms/step - loss: 0.3372 - acc: 0.8525 - mean_squared_error: 0.1058 - val_loss: 0.6332 - val_acc: 0.6880 - val_mean_squared_error: 0.2106
Epoch 115/500
